# 1. 패키지 불러오기

In [1]:
import folium
from folium.features import DivIcon
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json

# 2. 데이터 불러오기

In [2]:
seoul_panmae = pd.read_csv('/content/drive/MyDrive/Miniproject_No_1/0.Code&Data/datas/경쟁업체/소상공인시장진흥공단_상가(상권)정보_서울_202212.csv')
seoul_gu = pd.read_csv('/content/drive/MyDrive/Miniproject_No_1/0.Code&Data/datas/서울시 각 구별 위경도.csv')

FileNotFoundError: ignored

## 2-1. 데이터 정보 확인

In [ ]:
seoul_panmae.info()

In [ ]:
# 데이터 필터링 한 경우
seoul_panmae2 = seoul_panmae[seoul_panmae['상권업종대분류명'] == '음식']

* 해당 데이터의 경우 각 구별 음식 업체 현황을 확인하는 것으로써 별도의'count'컬럼을 주고 해당 값을 지도에 그렸습니다.
-> 본인 데이터에 맞게 수정 바랍니다.

## 2-2. count 계산 -> 데이터 프레임 만든 후 본 데이터에 넣어주기

In [ ]:
# count 계산
seoul_percentage = seoul_panmae2['시군구명'].value_counts()
seoul_percentage

In [ ]:
# Series -> DataFrame 화(본 데이터에 넣어주기 위해)
seoul_percentage2 = pd.DataFrame(seoul_percentage)
seoul_percentage2.reset_index(inplace=True)

In [ ]:
# (필요시) rename - 본 데이터와 가급적 동일한 컬럼명을 쓰는게 좋음
seoul_percentage2.rename(columns = {'시군구명' : 'count'}, inplace = True)
seoul_percentage2.rename(columns = {'index' : '시군구명'}, inplace = True)

In [ ]:
seoul_percentage2

In [ ]:
# 본 데이터에 구별 count한 값 합치기 -> 엑셀의 Vlookup과 동일한 원리라 생각하면됨.(엑셀 Vlookup 모르면 쩝... 수고스럽지만 한번 결과 해보는게 좋음)
seoul_panmae3 = seoul_panmae2.join(seoul_percentage2.set_index('시군구명')['count'], on='시군구명')
seoul_panmae3.info()

### ** 필요해서 만든 변수이니 본인의 데이터에 맞게 사용할 것

In [ ]:
seoul_panmae4 = seoul_panmae3.drop_duplicates(['시군구명'])
seoul_panmae4

# 3. 지도 그리기

## 3-1. geojson 파일 불러오기(단계 구분도는 선을 그려주기 위함임.)

In [ ]:
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

## 3-2. folium 패키지에 내장된 leaflet.js 지도 불러오기

In [ ]:
# 서울시 중심부의 위도, 경도 입니다.
center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center, zoom_start=10)

In [ ]:
seoul_panmae3['count']

In [ ]:
seoul_geo

# 3-3. folium 패키지에 내장된 choropleth로 단계구분도 그리기

In [ ]:
# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=seoul_geo, # geo_json 파일
    data=seoul_panmae3, # 해당 데이터 넣기
    columns=('시군구명', 'count'), # 해당하는 값
    key_on='feature.properties.name', # geo_json = key값(지역명 있는 json 키값 확인할 것)
    fill_color='BuPu', # 채우기 색
    legend_name='서울 업체수', # 범례 이름 넣기기
).add_to(m) # 위에 불러온 지도에 덧칠한다는 마인드드

# 맵 m을 출력합니다.
m

# 4. 지도에 텍스트 넣기

In [ ]:
seoul_gu.info()

## 4.1 지역명 + 위,경도 딕셔너리 만들기 -> 들어갈 텍스트의 위치 잡기 위함임.

In [ ]:
seoul_panmae5 = {}
for index in seoul_gu.index:
    seoul_panmae5[seoul_gu.loc[index, '시군구명'] ] = (seoul_gu.loc[index, "위도"], seoul_gu.loc[index, "경도"])

In [ ]:
seoul_panmae5

## 4.2 지도에 텍스트 넣기

In [ ]:
# (필요시) 데이터 약간 수정(넣을 텍스트를 위해)
seoul_panmae4["idx"] = seoul_panmae4["도로명주소"].str.split().str[1]
seoul_panmae4.set_index("idx", inplace=True)

In [ ]:
for key, value in seoul_panmae5.items():
    folium.map.Marker(
        # 위경도 위치
        [value[0], value[1]],  

        # DivIcon 을 사용
        # html 태그를 이용해서 text를 올릴 수 있음
        icon=DivIcon(
            # icon px 사이즈
            icon_size=(0, 0),
            # icon 좌 상단 위치 설정
            icon_anchor=(0, 0),

            # html 형식으로 text 추가
            # div 태그 안에 style 형식 추가
            html='<div\
                    style="\
                        font-size: 1.5rem;\
                        color: black;\
                        background-color:rgba(255, 255, 255, 0.2);\
                        width:120px;\
                        text-align:left;\
                        margin:0px;\
                    "><b>'
            + key + ': ' + str(seoul_panmae4.loc[key, 'count'])
        )).add_to(m)

In [ ]:
m